In [1]:
# import libraries and datasets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log2 as lg

# load the dataset

# TODO import correct test set
behavior = pd.read_csv("data/MINDsmall_dev/behaviors.tsv", sep="\t", header=None, names=["Impression ID", "User ID", "Time", "History", "Impressions"])
news = pd.read_csv("data/MINDsmall_dev/news.tsv", sep="\t", header=None, names=["News ID", "Category", "Subcategory", "Title", "Abstract", "URL", "Title Entities", "Abstract Entities", "Title Topics", "Abstract Topics"])

In [2]:
# Constants
theta = 0.5

In [ ]:
# Get all impressions for a user and add them to a df
def get_impressions(userID):
    test = []
    for index, row in behavior.iterrows():
        if row["User ID"] == userID:
            for impression in row["Impressions"].split(" "):
                imp = impression.split("-")
                test.append((imp[0], int(imp[1])))
    return pd.DataFrame(test, columns=["News ID", "Response"])

# Binarize score of all items in impressions for a user
def binarize_score(scored_data):
    binarized_data = []
    for index, row in scored_data.iterrows():
        if row["Score"] > theta:
            binarized_data.append((row["News ID"], 1, index))
        else:
            binarized_data.append((row["News ID"], 0, index))
    return pd.DataFrame(binarized_data, columns=["News ID", "Score", "index"])

# Join score and response tables
def create_evaluation_data(scored_data, userID):
    return binarize_score(scored_data).join(get_impressions(userID), how="left", on="News ID")

In [3]:
def DCGi(scored):
    if scored.iloc[0]["index"] == 0:
        dcgi = scored.iloc[0]["Response"]
    else:
        dcgi += scored.iloc[0]["Response"]/lg(scored.iloc[0]["index"]+1)

    for index, row in scored.iloc[1:].iterrows():
        dcgi += row["Response"]/lg(row["index"]+1)
    
    return dcgi



def nDCG(scored_data, userID):
    scored = create_evaluation_data(scored_data, userID)
    dcg = DCGi(scored)
    n = 1
    count = 2
    for index, row in scored.iloc[1:].iterrows():
        if row["Response"] == 1:
            n += 1/lg(count)
            count += 1
    return dcg/n
